# Python pandas project cz. I

Celem tego projeku jest pokazania umiejętności wykorzystywania bibioteki Pandas do manipulacji (data manipulation ) i czyszczenia danych (data cleaning)

In [ ]:
#W pierwszej kolejnosci importujemy Pandasa, tworzymy DataFrame i printujemy pierwsze 10 wynikow, aby zapoznać się z danymi

import pandas as pd
p='./ceny_produktow.csv'

df=pd.read_csv(p ,sep=';', decimal='.')
df.head(10) 

In [ ]:
# sprawdzamy unikatowe wartości dla kolumny Nazwa i widzimy, że jest Polska. Możemy ją odfiltorwać z DF, ponieważ potrzebujemy danych tylko z poszczególnych wojewódźtw

df['Nazwa'].unique() 

In [ ]:
#tworzenie warunku, aby odfiltorwać wartość POLSKA i nadpisanie DF już z warunkiem

w1=df['Nazwa'] != 'POLSKA'
df=df[w1] 
df

In [ ]:
#ten sam zabieg dla kolumny Rodzaje towarow. Widzimy ze jedna wartosc zawiera błędy ortograficzne i złą liczbę
df['Rodzaje towarów'].unique() 

In [ ]:
# Zmiana wartosci na prawidlowa przy użyciu loc
df.loc[df['Rodzaje towarów'] == 'jaja kóże śfierze - za 666szt.', 'Rodzaje towarów'] = 'jaja kurze swieze - za 12szt.' 

In [ ]:
#Ponowen sprawdzenie czy zmiana została zastosowana
df['Rodzaje towarów'].unique() 

In [ ]:
#Sprawdzenie jakie typy danych mamy w kolumnach. Widzimy ze Data nie jest Datetime tylko object (string)
df.info() 

In [ ]:
#Sprawdzamy jak zapisane sa daty i czy sa jakies dziwne daty --> '2099-13', '1888-0'
df['Data'].unique() 

In [ ]:
#Filtrowanie danych wraz z sprawdzeniem czy daty zostaly odflitrowane 
w2 = (df['Data'] != '2099-13') & (df['Data'] != '1888-0')  
df = df[w2]
df['Data'].unique() 
#df.reset_index(inplace=True)

In [ ]:
#Zamiana kolumny Data z typy string na Datetime oraz utworzenie nowych kolumn z rokiem, miesiacem oraz kwartalem
from datetime import datetime 

df["Data"] = pd.to_datetime(df['Data'])
df['Miesiac'] = df['Data'].dt.month
df['Rok'] = df['Data'].dt.year
df['Kwartal'] = df['Data'].dt.quarter



In [ ]:
#Zmiana nazw koloumn z polskich na angielskie i wyprintowanie 1 wiersza z nowymi kolumnami
df.columns=('province', 'product type', 'currency','ID', 'product line', 'value', 'date', 'month', 'year', 'quarter') 
df.head(1) 

In [ ]:
#Sprawdzamy unikatowe wartosci w currency i widzimy, że niektóre wartości są w EUR. Chcemy je zmienić na zł, ale również obliczyć nową wartość adekwatna do zł
df['currency'].unique() 

In [ ]:
#zapamietajmy że dla indexu 26 wartośc wynosiła 0.669880 EUR. Później sprawdzimy czy zostało to prawidłowo przeliczone
w3=df['currency'] =='EUR'
df[w3] 

In [ ]:
#Przyjmijmy dla obliczen ze 1EUR = 4.5 PLN
df.loc[w3, 'value'] = df.loc[w3, 'value'] * 4.5 

In [ ]:
#Szukamy wczesniej zapisanego przykladu dla indexu = 26 i widzimy ze wartosc zostala zmieniona. Pozostaje zamienic w kolumnie currency EUR na zl 
check = df.query('index == 26') 
check

In [ ]:
#Pozostaja nam zamiana wartosci z EUR na zł w kolumnie currency
df.loc[w3, 'currency'] = 'zł' 

In [ ]:
#Sprawdzenie czy w kolumnie currency mamy juz tylko zł
df['currency'].unique() 

In [ ]:
#Printujemy znowu dane i widzimy, ze w kolumanch product typ i product line nie zawsze mamy wartosci. 
df.head(10) 

In [ ]:
#Sprawdzamy czy jest to schemat, gdy wartość product type jest NaN to wartość w productline jest uzupełniona 
df['product type'].isna().value_counts()

In [ ]:
df.loc[df['product line'].isna(), 'product type'].isna().value_counts()

In [ ]:
#Tworzymy nową kolumnę product z zawartościami z kolumny product type
df['product']=df['product type']

In [ ]:
#Dodajemy do kolumny product wartości z product line, ale tylko tam gdzie product line nie jest puste
df.loc[df['product line'].notna(), 'product'] = df['product line']

In [ ]:
#Sprawdzamy czy jakiekolwiej wartości w nowej kolumnie są puste
df['product'].isna().value_counts()

In [ ]:
#Usuwamy duplikaty z tabeli i nadpisujemy od razu nasz DF
df=df.drop_duplicates()

In [ ]:
#Usuwamy dwie kolumny z których stworzyliśmy nową kolumne product
df.drop(['product type', 'product line'], axis=1, inplace=True)
df.head(10)


In [ ]:
#Zapisujemy nasze dane w formie CSV  jako Ceny_produktow_cleaned
p='./Ceny_produktow_cleaned.csv'
df.to_csv(p, sep=';',decimal='.', encoding='utf-8', index=False)